# Mixer correction
In this tutorial we will demonstrate the ability to compensate for output mixer non-idealities and observe the changes using an oscilloscope.

Mixer non-idealities can lead to unwanted spurs on the output (LO/RF/IF feedthrough and other spurious products) and they can be compensated by applying adjustments to the I/Q outputs: phase offset, gain ratio and DC offset. This solution applies to both baseband QCM/QRM products using external mixers as well as QCM-RF and QRM-RF products.

The tutorial is designed for Cluster QRM/QCM baseband. We will adjust all the parameters listed above and observe the changes to the I/Q outputs directly on an oscilloscope.

For QCM-RF and QRM-RF products, one can also refer to the 'mixer calibration' section of the tutorial on [RF-control](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/tutorials/q1asm_tutorials/basic/rf/rf_control.html#Mixer-calibration).

To run this tutorial please make sure you have installed and enabled ipywidgets: 
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

Setup
-----

First, we are going to import the required packages.

In [1]:
# Import ipython widgets
import json
import ipywidgets as widgets

# Set up the environment.
from IPython.display import display
from ipywidgets import fixed, interact, interactive
from qcodes import Instrument
from qblox_instruments import Cluster, PlugAndPlay

### Scan For Clusters

We scan for the available clusters on our network using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/tools.html#api-pnp) for more info).

In [2]:
# Scan for available devices and display
with PlugAndPlay() as p:
    # get info of all devices
    device_list = p.list_devices()

names = {dev_id: dev_info["description"]["name"] for dev_id, dev_info in device_list.items()}
ip_addresses = {dev_id: dev_info["identity"]["ip"] for dev_id, dev_info in device_list.items()}

# create widget for names and ip addresses
connect = widgets.Dropdown(
    options=[
        (names[dev_id] + " @" + ip_addresses[dev_id], dev_id) for dev_id in device_list.keys()
    ],
    description="Select Device",
)
display(connect)

Dropdown(description='Select Device', options=(('Marketing-Cluster @192.168.1.170', '00015_2247_002'),), value…

### Connect to Cluster

We now make a connection with the Cluster selected in the dropdown widget. We also define a function to find the modules we're interested in. We select the readout and control module we want to use.

In [3]:
# Connect to device
dev_id = connect.value
# Close the chosen QCodes instrument as to prevent name clash.

# PlugAndPlay assigns an instrument name with a hyphen. For example 'pulsar-qrm' as a string for instrument name.
# QCodes instrument class cannot handle hyphens, and changes them to underscore '_'.
# However, this happens only once with a warning and never again, so code does not work. This line does the automation for it.
names[dev_id] = names[dev_id].replace("-", "_")

try:
    Instrument.find_instrument(names[dev_id]).close()
except KeyError:
    pass

cluster = Cluster(name=names[dev_id], identifier=ip_addresses[dev_id])

print(f"{connect.label} connected")
print(cluster.get_system_state())

Marketing-Cluster @192.168.1.170 connected
Status: OKAY, Flags: NONE, Slot flags: NONE


In [4]:
def select_module_widget(device, select_all=False, select_rf_type: bool = True):
    """Create a widget to select modules of a certain type

    default is to show only QRM baseband

    Args:
        devices : Cluster we are currently using
        select_all (bool): ignore filters and show all modules
        select_rf_type (bool): filter RF/baseband
    """
    options = [[None, None]]

    for module in device.modules:
        if module.present():
            if select_all or module.is_rf_type == select_rf_type:
                options.append(
                    [
                        f"{device.name} "
                        f"{module.short_name} "
                        f"({module.module_type}{'_RF' if module.is_rf_type else ''})",
                        module,
                    ]
                )
    widget = widgets.Dropdown(options=options)
    display(widget)

    return widget


print("Select QCM or QRM module from the available ones:")
select_module = select_module_widget(cluster, select_rf_type=True)

Select QCM or QRM module from the available ones:


Dropdown(options=([None, None], ['Marketing_Cluster module4 (QCM_RF)', <QcmQrm: Marketing_Cluster_module4 of C…

In [5]:
# Connect to the cluster QCM/QRM module
module = select_module.value
print(f"{module} connected")
print(cluster.get_system_state())

<QcmQrm: Marketing_Cluster_module8 of Cluster: Marketing_Cluster> connected
Status: OKAY, Flags: NONE, Slot flags: NONE


### Reset the Cluster

We reset the Cluster to enter a well-defined state. Note that resetting will clear all stored parameters, so resetting between experiments is usually not desirable.

In [8]:
cluster.reset()
print(cluster.get_system_state())

Status: OKAY, Flags: NONE, Slot flags: NONE


### Setup Sequencer <a id='setup_seq'></a>


The easiest way to view the influence of the mixer correction is to mix the NCO sin and cos with I and Q values of 1 (fullscale). The instrument output would be simple sinusoids with a 90[deg] phase offset and identical amplitude.

We use sequencer 0 to set I and Q values of 1 (fullscale) using DC offset and we mix those with the NCO signals.

In [9]:
# Program sequence we will not use.
sequence = {"waveforms": {}, "weights": {}, "acquisitions": {}, "program": "stop"}
with open("sequence.json", "w", encoding="utf-8") as file:
    json.dump(sequence, file, indent=4)
    file.close()
module.sequencer0.sequence(sequence)

# Program fullscale DC offset on I & Q, turn on NCO and enable modulation.
module.sequencer0.offset_awg_path0(1.0)
module.sequencer0.offset_awg_path1(1.0)
module.sequencer0.nco_freq(10e6)
module.sequencer0.mod_en_awg(True)

Control sliders
-----

Create control sliders for the parameters described in the introduction. Each time the value of a parameter is updated, the sequencer is automatically stopped from the embedded firmware for safety reasons and has to be manually restarted.

The sliders cover the valid parameter range. If the code below is modified to input invalid values, the firmware will not program the values.

Please connect the I/Q outputs ($\text{O}^{[1-2]}$) to an oscilloscope and set to trigger continuously on the I channel at 0V. Execute the code below, move the sliders and observe the result on the oscilloscope.

In [10]:
def set_offset_I(offset_I):
    module.out0_offset(offset_I)
    module.arm_sequencer(0)
    module.start_sequencer(0)


def set_offset_Q(offset_Q):
    module.out1_offset(offset_Q)
    module.arm_sequencer(0)
    module.start_sequencer(0)


def set_gain_ratio(gain_ratio):
    module.sequencer0.mixer_corr_gain_ratio(gain_ratio)
    module.arm_sequencer(0)
    module.start_sequencer(0)


def set_phase_offset(phase_offset):
    module.sequencer0.mixer_corr_phase_offset_degree(phase_offset)
    module.arm_sequencer(0)
    module.start_sequencer(0)


I_bounds = module.out0_offset.vals.valid_values
interact(
    set_offset_I,
    offset_I=widgets.FloatSlider(
        min=I_bounds[0], max=I_bounds[1], step=0.01, value=0.0, description="Offset I:"
    ),
)

Q_bounds = module.out1_offset.vals.valid_values
interact(
    set_offset_Q,
    offset_Q=widgets.FloatSlider(
        min=Q_bounds[0], max=Q_bounds[1], step=0.01, value=0.0, description="Offset Q:"
    ),
)

# The gain ratio correction is bounded between 1/2 and 2
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatLogSlider(
        min=-1, max=1, step=0.1, value=1.0, base=2, description="Gain ratio:"
    ),
)

ph_bounds = module.sequencer0.mixer_corr_phase_offset_degree.vals.valid_values
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(
        min=ph_bounds[0], max=ph_bounds[1], step=1.0, value=0.0, description="Phase offset:"
    ),
)

AttributeError: 'QcmQrm' object and its delegates have no attribute 'out0_offset'

When tuning the DC offset you might notice that the signal starts "clipping". This is caused by the fact that we are already at full-scale, thus any offset takes our signal out of its dynamic range. 

When this happens, the output LEDs on the module turn orange. This, and other LEDs states, are explained in the [troubleshooting](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/cluster/troubleshooting.html) guide.

### Reset the Cluster

We reset the Cluster to enter a well-defined state. Note that resetting will clear all stored parameters, so resetting between experiments is usually not desirable.

In [11]:
cluster.reset()
print(cluster.get_system_state())

Status: OKAY, Flags: NONE, Slot flags: NONE
